Get started with required imports:

In [ ]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import requests, os, time, lxml

from dotenv import load_dotenv

from bs4 import BeautifulSoup

Load API key from .env and run a sanity check to confirm value is present.

In [ ]:
API_KEY = os.getenv("NPS_API_KEY")

print(API_KEY is not None)

I'm going to use the dataset to create a dataframe. The data we need is on the second sheet, so I need to specify. I'll also look at the first few rows to make sure this is what I want.

In [ ]:
df = pd.ExcelFile(r"../data/NPS-Mortality-Data-CY2007-to-CY2024-Released-August-2024.xlsx")
df = df.parse(sheet_name="CY2007-Present Q2")
df.head()

Use .info() to get a look at the column names, datatypes, and counts.

In [ ]:
df.info()

Print a list of the columns that may be easier to reference.

In [ ]:
print(df.columns)

I'm going to look more closely at the "Cause of death" and "Cause of Death Group \n..." columns to see if this data is redundant, or if I need to use both in my analysis.

Check if all values in all rows and columns are the same.

In [ ]:
(df["Cause of Death"] == df["Cause of Death Group \n(Used in the NPS Mortality Dashboard) "]).all()

Since the are not duplicates, I'm going to build a mask to count how many mismatches are in the data.

In [ ]:
mask_mismatch = df["Cause of Death"] != df["Cause of Death Group \n(Used in the NPS Mortality Dashboard) "]
print("Total rows:", len(df))
print(mask_mismatch.sum())

I'm going to look at some of the mismatches to see what the differences are.

In [ ]:
df.loc[mask_mismatch, ["Cause of Death", "Cause of Death Group \n(Used in the NPS Mortality Dashboard) "]].head()

It seems like I will be able to use the "Cause of Death" column and ignore the group column for now. I'm going to look more closely at the cause of death column now.

In [ ]:
df.groupby("Cause of Death").size()

There are some obvious duplicates in that list, so I'm going to normalize the causes to try to catch duplicates. First I'm going to see how many unique values there are now.

In [ ]:
df["Cause of Death"].nunique()

In [ ]:
df["Cause of Death"] = df["Cause of Death"].str.strip().str.lower()

I want to see how many unique values I have now.

In [ ]:
df["Cause of Death"].nunique()

I'm going to look at a list of the unique values just to see what I have since there aren't that many. I'll sort them in alphabetical order so it will be a little easer to see duplicates.

In [ ]:
print(sorted(df["Cause of Death"].unique()))

Looking over the columns, I don't see any duplicates or names that should be changed right now. I'm going to change them back to have an uppercase character.

In [ ]:
df["Cause of Death"] = df["Cause of Death"].str.title()
print(df["Cause of Death"])

Now that I know what column to use and I have eliminated case mismatches, I'm going to build a plot to just look at the causes of death.

In [ ]:
cause_counts = df["Cause of Death"].value_counts()

cause_counts.plot(
    kind="pie",
    autopct="%1.1f%%",
    figsize=(8,8),
    startangle=90
)

plt.ylabel("")
plt.title("Cause of Death")
plt.show()



There are lots of less frequent causes that confuse this plot. I'll do some adjustments to make it more meaningful/easy to read.

I want to show the major causes of death. This will calculate the percentage of deaths attributed to each cause, and also group causes accounting for <5% of deaths be combined into an "other" category to avoid cluttering the plot. The variables defined here will be used in the plot.

In [ ]:
cause_counts = df["Cause of Death"].value_counts(dropna=False)
percentages  = cause_counts / cause_counts.sum() * 100

major_causes = percentages[percentages >= 5].copy()
other_total  = percentages[percentages < 5].sum()
if other_total > 0:
    major_causes.loc["Other"] = other_total

I'm going to define my color palette to be used with this plot. I'm also going to set the colors up be the same for both medical cause deaths.

The project’s color palette (hex codes and ordered list) was assisted by ChatGPT to ensure consistent formatting and to streamline selection and presentation.

After generation, I manually verified each hex code and palette entry by comparing them against the reference palette published here:  
<https://siegal.bio.nyu.edu/color-palette/>

No discrepencies were found other than the omission of black, which was intentional to avoid readability issues.

In [ ]:
okabe_ito = [
    "#E69F00",
    "#56B4E9",
    "#009E73",
    "#F0E442",
    "#0072B2",
    "#D55E00",
    "#CC79A7", 
]
OTHER_GREY = "#9e9e9e"

def color_for_label(lbl):
    s = str(lbl).strip().lower()
    if s == "other":
        return OTHER_GREY

    if "medical" in s:
        return "#E69F00"  # orange
    return None



Now that the colors are defined, I'll build a for loop to cycle through the unused options.

In [ ]:
used = {c for c in [color_for_label(x) for x in major_causes.index] if c}
remaining = [c for c in okabe_ito if c not in used]
colors = []
idx = 0
for lbl in major_causes.index:
    c = color_for_label(lbl)
    if c is None:
        c = remaining[idx % len(remaining)]
        idx += 1
    colors.append(c)

Now I'm going to create a pie chart that shows the major causes of death.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
wedges, texts, autotexts = ax.pie(
    major_causes.values,
    labels=major_causes.index,
    colors=colors,
    autopct="%1.1f%%",
    wedgeprops={"linewidth": 1.0, "edgecolor": "white"},
    startangle=90
)
for t in texts:
    t.set_color("#111111")
    t.set_fontsize(11)

for t in autotexts:
    t.set_fontsize(11)
    t.set_color("#111111")
  

ax.set_title("Major Causes of Death in NPS Territories")

plt.tight_layout()
plt.show()


Drowning is the most frequent cause of death, so I'm going to look at that more closely to see if there are any common threads. I'm going to look at the number of drownings in each territory/park to see if there are any outliers. First, I'm going to normalize the park names to check for any case mismatches.

In [ ]:
df["Park Name"].nunique()

In [ ]:
df["Park Name"] = df["Park Name"].str.strip().str.lower()

In [ ]:
df["Park Name"].nunique()

There was no change to the number of unique strings, so it doesn't seem like there are mismatches in this column. I'm going to change back to original caps formatting.

In [ ]:
df["Park Name"] = df["Park Name"].str.title()
print(df["Park Name"])

I want to see if there are any null values in these columns.

In [ ]:
df.isna().sum()

I know there are no missing values, so I'm going to quickly look at the drowning deaths per territory.

In [ ]:
#Filter for drowning
drowning_df = df[df["Cause of Death"] == "Drowning"]

#Count drowning deaths in parks
drowning_counts = drowning_df["Park Name"].value_counts()

#Create bar graph with data
plt.figure(figsize=(14,6))
drowning_counts.plot(kind="bar")

plt.title("Drowning Deaths by National Park")
plt.xlabel("Park Name")
plt.ylabel("Number of Drowning Deaths")
plt.xticks(rotation=90)

plt.show()

It looks like there are some outliers, but you can't really see more than that since there are so many entries. I'm going to create another, neater plot with the top 15 parks.

In [ ]:
top_drowning = drowning_counts.head(15).sort_values()

top_drowning.plot(kind="barh")
plt.title("Top 15 Parks by Drowning Deaths")
plt.xlabel("Drowning Deaths")
plt.ylabel("")
plt.xticks(rotation=0)
plt.show()

I'm not working with the age range values right now, but I'm going to go ahead and look at those nulls since the rest of the dataset is complete.

In [ ]:
print(df["Age Range"].unique())

So, there's problems. Unintentional, nan, and not reported should all be unknown values. 0 - 14 and 0-14 seem like duplicates. I need to fix the ranges and make as many as possible numeric values. I'm going to start by combining the unknown values and counting them.

In [ ]:
unknown_age = {"Not Reported", "Unintentional", "Unknown", None, np.nan}

df["age_range_clean"] = df["Age Range"].replace(list(unknown_age), "Unknown")

print(df["age_range_clean"].value_counts())

Now I'm going to normalize the spacing to combine the duplicate age ranges. I'm going to normalize the dashes

In [ ]:
df["age_range_clean"] = (
    df["age_range_clean"]
    .str.replace(r"\s*-\s*", "-", regex=True)  # normalize dashes
    .str.strip()
)

I'm going to double check that the values combined correctly.

In [ ]:
print(df["age_range_clean"].value_counts())

I may have to revisit 65+, depending on what type of analysis I want to do, but for now I'm going to leave it as is and comvert the numeric ranges into min and max values, and calculate the midpoint. I'll use nan value for the max age in the 65+ column. Being numeric values instead of strings will prepare the data for any by age statistical analysis I may want to do later.

In [ ]:
ranges = df["age_range_clean"].str.extract(r"(?P<min>\d+)-(?P<max>\d+)")
df["age_min"] = ranges["min"].astype(float)
df["age_max"] = ranges["max"].astype(float)

mask_plus = df["age_range_clean"].str.endswith("+", na=False)
df.loc[mask_plus, "age_min"] = df.loc[mask_plus, "age_range_clean"].str[:-1].astype(float)
df.loc[mask_plus, "age_max"] = np.nan 

df["age_mid"] = (df["age_min"] + df["age_max"]) / 2

Confirm that the columns have been added as numeric values

In [ ]:
df.dtypes

One more sanity check -- I want to see the values and make sure that the min/max/mid ages are correct, and the NaN values are as expected.

In [ ]:
print(df[["age_range_clean","age_min","age_max","age_mid"]].drop_duplicates())

I know that I want to use data from other NPS datasets along with this information. I also know that there are 3 major sources (NPS Mortality Dataset, IRMA API, and NPS Data API) and the park name is inconsistent across all 3. The API datasets both use a 4 letter code that is  consistent. I'm going to use one of the APIs to match the names of the parks in the mortality dataset to the appropriate park code so that I may query either database for info with the 4 letter code.

BASE = "https://developer.nps.gov/api/v1/parks"
params = {"limit": 50, "start": 0, "api_key": API_KEY}
r = requests.get(BASE, params=params, timeout=30)
print(r.status_code) 

Requesting list of keys for data returned so I can see how to filter for the park code and the full name.

BASE = "https://developer.nps.gov/api/v1/parks"
params = {"limit": 50, "start": 0, "api_key": API_KEY}
r = requests.get(BASE, params=params, timeout=30)
data = r.json()["data"]
if data:
    print(list(data[0].keys()))

I want the "fullName" and "parkCode" for each row. I'm going to craft an API call to build this dataframe.

def fetch_all_parks(api_key: str, page_size: int = 50, pause: float = 0.2) -> pd.DataFrame:
    base = "https://developer.nps.gov/api/v1/parks"
    start = 0
    rows = []

#Create a loop that will continue until there is no data or the data returned is < the page size
    while True:
        params = {"limit": page_size, "start": start, "api_key": api_key}
        resp = requests.get(base, params=params, timeout=30)
        resp.raise_for_status()
        #convert the response from JSON text to Python dictionary
        payload = resp.json()

        data = payload.get("data", [])
        if not data:
            break
        
        #The default parks response has more info than I need, so I'm going to filter just for the name and park code
        for item in data:
            rows.append({
                "fullName": item.get("fullName"),
                "parkCode": item.get("parkCode")
            })

        if len(data) < page_size:
            break

        start += page_size
        time.sleep(pause)

    return pd.DataFrame(rows).drop_duplicates().reset_index(drop=True)

df_parks = fetch_all_parks(API_KEY, page_size=50)
df_parks.head()

Saving the data with park names and park codes to a .csv file

In [ ]:
print("Number of parks:", len(df_parks))
df_parks.to_csv("nps_parks_fullName_parkCode.csv", index=False)

Now we'll examine the new dataframe.

In [ ]:
df_parks.info()

In [ ]:
df_parks.tail()

In [ ]:
df_parks["fullName"].duplicated().sum()

In [ ]:
df_parks["parkCode"].duplicated().sum()

Now that I have the names and the park codes, I'm going to build a dictionary with the df_parks dataframe that I can use to add the parkCode to our existing dataframe

In [ ]:
code_map = dict(zip(df_parks["fullName"], df_parks["parkCode"]))

#Make a copy of the mortality dataframe so we aren't modifying original
df_with_codes = df.copy()

#Add columns for parkCode and Park Name
df_with_codes["parkCode"] = df_with_codes["Park Name"].map(code_map)

#Merge data from df_parks into new columns for exact matches
name_map = dict(zip(df_parks["fullName"], df_parks["fullName"]))
df_with_codes["fullName_official"] = df_with_codes["Park Name"].map(name_map)

#Print info about what was merged, what is missing
total = len(df_with_codes)
matched = df_with_codes["parkCode"].notna().sum()
print(f"Total rows: {total}")
print(f"Matched rows: {matched}")
print(f"Unmatched rows: {total - matched}")

unmatched_names = (
    df_with_codes.loc[df_with_codes["parkCode"].isna(), "Park Name"]
    .dropna()
    .drop_duplicates()
    .sort_values()
)

print("\nUnique unmatched park names:")
for nm in unmatched_names:
    print("-", nm)

Since there aren't that many, I'm going to create a manual map to add the park codes using the NPS.gov site as a reference (https://www.nps.gov/articles/000/historic-listing-of-nps-park-codes.htm)

The `manual_crosswalk` table used in this notebook was generated with the assistance of ChatGPT to minimize copy/paste and encoding errors in character-specific strings (e.g., *Haleakalā*, *Hawaiʻi*, *Wrangell–St. Elias*, “&” vs “and”, etc.). This helped prevent merge failures when normalizing park names.

 After generation, I manually verified every entry by comparing each `fullName`/`parkCode` against the official NPS website for accuracy. Any mismatches were corrected in the crosswalk file before use in the analysis.

The resulting file is saved as `parkname_manual_crosswalk.csv` and versioned in the repository. If an NPS unit is renamed or redesignated in the future, the crosswalk should be reviewed and updated accordingly.

In [ ]:
manual_crosswalk = pd.DataFrame([
    ("Big South Fork National River And Recreation Area", "Big South Fork National River & Recreation Area", "biso"),
    ("Canyon De Chelly National Monument",                "Canyon de Chelly National Monument",            "cach"),
    ("Castillo De San Marcos National Monument",          "Castillo de San Marcos National Monument",      "casa"),
    ("George Washington Birthplace",                      "George Washington Birthplace National Monument","gewa"),
    ("Haleakala National Park",                           "Haleakalā National Park",                       "hale"),
    ("Hawaii Volcanoes National Park",                    "Hawaiʻi Volcanoes National Park",               "havo"),
    ("Jean Lafitte National Historical Park & Preserve",  "Jean Lafitte National Historical Park and Preserve","jela"),
    ("Kaloko-Honokohau National Historical Park",         "Kaloko-Honokōhau National Historical Park",     "kaho"),
    ("National Mall & Memorial Parks",                    "National Mall and Memorial Parks",              "nama"),
    ("New River Gorge National River",                    "New River Gorge National Park and Preserve",    "neri"),  # redesignated; code remains NERI
    ("Not Reported",                                      "Not Reported",                                  None),
    ("President'S Park (White House)",                    "President's Park (White House)",                "whho"),
    ("Presidio Of San Francisco",                         "Presidio of San Francisco",                     "prsf"),
    ("Redwood National And State Parks",                  "Redwood National and State Parks",              "redw"),
    ("Suitland",                                          "Suitland Parkway (under National Capital Parks–East)", "nace"),  # official unit under NACE
    ("Wilson'S Creek National Battlefield",               "Wilson's Creek National Battlefield",           "wicr"),
    ("Wrangell - St Elias National Park & Reserve",       "Wrangell–St. Elias National Park & Preserve",   "wrst"),
    ("Yorktown Battlefield Part Of Colonial National Historical Park", "Yorktown Battlefield, Colonial National Historical Park", "colo"),  # site rolls up under COLO
], columns=["Park Name", "fullName_manual", "parkCode_manual"])


I'll save the manual_croswalk as a .csv, then continue to use the mask to add codes, and then show anything remaining that is still missing a code.

In [ ]:

manual_crosswalk.to_csv("parkname_manual_crosswalk.csv", index=False)

# Build lookups (unique index)
code_lookup = manual_crosswalk.set_index("Park Name")["parkCode_manual"]
name_lookup = manual_crosswalk.set_index("Park Name")["fullName_manual"]

df_with_codes["parkCode"] = df_with_codes["parkCode"].fillna(
    df_with_codes["Park Name"].map(code_lookup)
)

# Now recompute the mask for *currently* missing codes
mask_missing_code = df_with_codes["parkCode"].isna() & df_with_codes["Park Name"].notna()

df_with_codes.loc[mask_missing_code].head()

Sanity check

In [ ]:
na_count = df_with_codes["parkCode"].isna().sum()
print(na_count)

Looking at the rows that are missing, they are all for unreported park names. This is expected since there can be no park code match. I don't need to do anything with these rows because I'm looking specifically at drowning deaths so the won't be included in my dataset. They may need to be excluded for different types of analysis, though.

Now that I have the codes I'll need to get data from NPS API systems and I've cleaned the rows I will be focusing on, I'm going to save this dataframe as a new .csv.

In [ ]:
#df_with_codes.to_csv("df_with_codes.csv", index=False)

Sanity check -- let's look at our new dataframe and see if any more cleaning needs to be done before querying for visitation data.

In [ ]:
df_with_codes.dtypes

I haven't really loked at the "outcome" column, so we'll look more closely there.

In [ ]:
df_with_codes["Outcome"].nunique()

In [ ]:
df_with_codes["Outcome"].unique()

In [ ]:
df_with_codes["Outcome"].value_counts()

This doesn't add anything to our dataset. It seems to be a case mismatch, and we already know that each row is a "Fatal Injury". I'm going to drop the entire column.

In [ ]:
df_with_codes.drop("Outcome", axis=1, inplace=True)

In [ ]:
df_with_codes.dtypes

To keep track of what columns we've cleaned, I'll use this list:

CLEANED
Park Name                                                                
Cause of Death                                                           
_Cause of Death Group \n(Used in the NPS Mortality Dashboard)             
Age Range                                                                
age_range_clean                                                          
age_min                                                                 
age_max                                                                 
age_mid                                                                 
parkCode                                                                 
fullName_official                                                        

TO BE CLEANED
Incident Date                                                    
Intent                                                                  
Sex                                                                      
Activity                                                                 

I'll look at the Incident Date more closely.

In [ ]:
df_with_codes["Incident Date"].head()

These are in correct datetime format and I know there are no missing values, so I'll move on.

To keep track of what columns we've cleaned, I'll use this list:

CLEANED
Park Name                                                                
Cause of Death                                                           
_Cause of Death Group \n(Used in the NPS Mortality Dashboard)             
Age Range                                                                
age_range_clean                                                          
age_min                                                                 
age_max                                                                 
age_mid                                                                 
parkCode                                                                 
fullName_official                                                        
Incident Date 

TO BE CLEANED                                                 
Intent                                                                  
Sex                                                                      
Activity                                                                 

We'll look at the intent column:

In [ ]:
df_with_codes["Intent"].head()

In [ ]:
df_with_codes["Intent"].value_counts()

I want to see if the "Medical" rows all correspond with the various medical causes of death. I'm going to normalize the Intent and Cause of Death columns and then compare to how many I would expect with the medical categories (3).

In [ ]:
df_with_codes["Intent_norm"] = df_with_codes["Intent"].astype(str).str.strip().str.casefold()
df_with_codes["Cause_norm"]  = df_with_codes["Cause of Death"].astype(str).str.strip().str.casefold()

# 1) First, see what causes appear under Intent == "Medical"
medical_causes_found = (
    df_with_codes.loc[df_with_codes["Intent_norm"] == "medical", "Cause_norm"]
      .dropna()
      .unique()
)
print("Causes seen for Intent == Medical:", medical_causes_found)
print("Count:", len(medical_causes_found))

This is what I expected. I'm just going to double check that the number of medical intent fields matches the sum of the three types of medical cause fields.

In [ ]:
medical_causes = [
    "Medical - Not During Physical Activity",
    "Medical - During Physical Activity",
    "Medical - Unknown"
]

df_with_codes["Cause of Death"].value_counts().loc[medical_causes]

There are 640 medical causes, and 640 medical intents, so I believe they corrospond correctly. 

To keep track of what columns we've cleaned, I'll use this list:

CLEANED
Park Name                                                                
Cause of Death                                                           
_Cause of Death Group \n(Used in the NPS Mortality Dashboard)             
Age Range                                                                
age_range_clean                                                          
age_min                                                                 
age_max                                                                 
age_mid                                                                 
parkCode                                                                 
fullName_official                                                        
Incident Date 
Intent  

TO BE CLEANED                                                 
                                                             
Sex                                                                      
Activity                                                                 

We'll look at the Sex column next.

In [ ]:
df_with_codes["Sex"].value_counts()

These are the values we would expect and there are no missing values. No additional work is necessary.

To keep track of what columns we've cleaned, I'll use this list:

CLEANED
Park Name                                                                
Cause of Death                                                           
_Cause of Death Group \n(Used in the NPS Mortality Dashboard)             
Age Range                                                                
age_range_clean                                                          
age_min                                                                 
age_max                                                                 
age_mid                                                                 
parkCode                                                                 
fullName_official                                                        
Incident Date 
Intent  
Sex

TO BE CLEANED                                                 
                                                                    
Activity                                                                 

Now we'll look more closely at the Activity data.

In [ ]:
df_with_codes["Activity"].head()

I'm going to look at the number of unique values, normalize, and compare just like I did with the Cause of Death column.

In [ ]:
df_with_codes["Activity"].nunique()

In [ ]:
df_with_codes["Activity"] = df["Activity"].str.strip().str.lower()


In [ ]:
df_with_codes["Activity"].nunique()

Now I'll add the caps back in.

In [ ]:
df_with_codes["Activity"] = df_with_codes["Activity"].str.title()

In [ ]:
df_with_codes["Activity"].value_counts()

In [ ]:
df_with_codes["Activity"].sort_values().unique()

I can combine "Rock Scrambeling" and "Rock Scrambling". I considered combining "Not Reported" and "Undetermined, but chose not to at this time because they do convey different information.

In [ ]:
df_with_codes["Activity"] = df_with_codes["Activity"].replace(
    "Rock Scrambeling", "Rock Scrambling"
)

In [ ]:
df_with_codes["Activity"].sort_values().unique()

That's the last "To Be Cleaned" column. I'm going to create a new csv with the cleaned columns.

In [ ]:
#df_with_codes.to_csv("df_with_codes_cleaned.csv", index=False)

I'm going to use this API call to obtain the total number of visitations for both recreation and nonrecreation in the month the death occured.

IRMA is looking for case sensitive all caps in the parkCode field, so I'm going to change that before building the API. I do have missing values in this column, so I'm going to 

In [ ]:
df_with_codes["parkCode"] = (df_with_codes["parkCode"].str.upper())


In [ ]:
df_with_codes["parkCode"].head()

I'm going to look at a single row and build an API call to get the information for a single, manual entry. 

In [ ]:
df_with_codes.head(1)

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"
params = {
    "unitCodes": "GLCA",
    "startMonth": 1, "startYear": 2007,
    "endMonth": 1,   "endYear": 2007,
    "format": "json",
}

r = requests.get(BASE, params=params, headers={"Accept": "application/json"}, timeout=30)

Check for 200

In [ ]:
print(r.status_code) 

I want to see the data that was returned.

In [ ]:
data = r.json()
print(data)

Now that I know how to build the API call, I want to determine the data I need. I want to get the monthly visitation statistics for each park in the mortality dataset, for the duration of the mortality dataset. I can then selectively pull from that dataset as needed for analyisis.

In [ ]:
date_col = "Incident Date"
code_col = "parkCode"

df_with_visits = df_with_codes.copy()

In [ ]:
df_with_visits.head()

In [ ]:
#Create the new dataframe to store this data
df_with_visits = df_with_visits.dropna(subset=["parkCode"])

#Define the chronologically first and last index in the dataset
i_first = df_with_visits[date_col].idxmin()
i_last = df_with_visits[date_col].idxmax()

first_row = df_with_visits.loc[i_first, [date_col, code_col]]
last_row  = df_with_visits.loc[i_last,  [date_col, code_col]]

print("First death:", first_row.to_dict())
print("Last death:",  last_row.to_dict())

Sanity check -- make sure the missing parkCode values were dropped.

In [ ]:
df_with_visits["Incident Date"].isna().sum()

Now that I know the first and last month to collect and have verified the API, I'm going to build an API call that collects the monthly visitation stats for every month 01/2007-06/2024.

BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"
params = {
    "unitCodes": "GLCA",
    "startMonth": 1, "startYear": 2007,
    "endMonth": 6,   "endYear": 2024,
    "format": "json",
}

r = requests.get(BASE, params=params, headers={"Accept": "application/json"}, timeout=30)

In [ ]:
data = r.json()
print(data)

I've got the call to collect the data for an individual park, but I want to collect all the parks in my dataset. 

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"

# 1) get unique park
all_codes = (
    df_with_visits["parkCode"]
    .dropna()
    .astype(str)
    .str.upper()
    .unique()
    .tolist()
)

def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

CODES_PER_CALL = 50

all_results = []

with requests.Session() as s:
    for chunk in chunk_list(all_codes, CODES_PER_CALL):
        params = {
            "unitCodes": ",".join(chunk),
            "startMonth": 1,
            "startYear": 2007,
            "endMonth": 6,
            "endYear": 2024,
            "format": "json",
        }
        r = s.get(BASE, params=params, headers={"Accept": "application/json"}, timeout=30)
        r.raise_for_status()
        data = r.json()
        all_results.extend(data)

visits_df = pd.DataFrame(all_results)


I want to view the first few rows of the new visits dataframe.

In [ ]:
visits_df.head()

In [ ]:
visits_df.shape

In [ ]:
visits_df.info

I'm going to save this visitation information to a .csv

In [ ]:
visits_df.to_csv("NPS_visitation_data.csv", index=False)

Next, I'll check to see if there are any missing values in any of the fields.

In [ ]:
visits_df.isna().sum()


Nothing is missing, so I'm going to check for completeness. If I received all the data I request, the number of rows for each UnitCode will be the same for each park.

In [ ]:
#count the number of rows for each park code
counts = visits_df["UnitCode"].value_counts()
counts

In [ ]:
counts.nunique()

There are 5 unique counts for unit codes, so I want to know why I don't have the same for the outliers. I'm going to define the one that is most common to be the expected, and then look for the other values.

In [ ]:
#Define the most common row using mode
expected = counts.mode()[0]

#show anything that doesn't match
mismatch = counts[counts != expected]

#Show the result
mismatch

I'm going to make sure it wasn't a problem with my call to the api and make another call requesting all the data for just those park units.

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"

list_missing_codes = ["MISS", "SACR", "PAGR", "FRST"]
mismatch_codes = (pd.Series(list_missing_codes)
    .dropna()
    .astype(str)
    .str.upper()
    .unique()
    .tolist()
)

def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

CODES_PER_CALL = 50

mismatch_all_results = []

with requests.Session() as s:
    for chunk in chunk_list(mismatch_codes, CODES_PER_CALL):
        params = {
            "unitCodes": ",".join(chunk),
            "startMonth": 1,
            "startYear": 2007,
            "endMonth": 6,
            "endYear": 2024,
            "format": "json",
        }
        r = s.get(BASE, params=params, headers={"Accept": "application/json"}, timeout=30)
        r.raise_for_status()
        data = r.json()
        mismatch_all_results.extend(data)

new_mismatch_data = pd.DataFrame(mismatch_all_results)

Now that I have the data, I'm going to count the rows for each code like I did before.

In [ ]:
mismatch_counts = new_mismatch_data["UnitCode"].value_counts()
mismatch_counts


In [ ]:
frst_rows = new_mismatch_data[new_mismatch_data["UnitCode"] == "FRST"]

I want to look at some of the rows to see what we do have. FRST is the park unit with the least rows, so I'll start there.

In [ ]:
frst_rows

I want to confirm that there really is no data prior to the first month I have, which is 01/2023. I'm going to make one additional API call to see what the earliest record is just to confirm this isn't due to an error in my previous API call to get this info.

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"
params = {
    "unitCodes": "FRST",
    "startYear": 2010,
    "endYear": 2025,
    "format": "json",
}
r = requests.get(BASE, params=params)
data = r.json()

# find earliest record
earliest = min([f"{d['Year']}-{d['Month']:02}" for d in data])
earliest


I've confirmed that this is the earliest data available and I've done a google search to see if there is anything I can find that would explain why the data isn't available. I haven't found anything. 

I'm going to add the visitation data into my cleaned df with codes, so I'm just going to look at the first few rows to refresh my memory on my column names.

In [ ]:
df_with_visits.head()

I see a few things that I can go ahead and clean up before I add the new data. I'm going to get rid of the original Intent and Cause fields and replace them with the normalized data. I'll get rid of the old Age Range Data and rename the cleaned data. I still have them in my orignal dataset if I need to reference the original version, and they're redundant here.

In [ ]:
df_with_visits = df_with_visits.drop(columns=["Age Range", "Intent", "Cause of Death"])

In [ ]:
df_with_visits.head()

In [ ]:
visits_df.head()

I'm going to pull the month and year out of my Incident Date datetime field so that I can use those as a key to match my visitation data and see if I'm missing data for any of the incidents in my mortality dataset.

In [ ]:
df_with_visits["Month"] = df_with_visits["Incident Date"].dt.month
df_with_visits["Year"]  = df_with_visits["Incident Date"].dt.year
df_with_visits.head()

Now I'm going to merge the dataframes

In [ ]:
df_with_visits = df_with_visits.merge(
    visits_df,
    how="left",
    left_on=["parkCode", "Month", "Year"],
    right_on=["UnitCode", "Month", "Year"]
)
df_with_visits

I want to check and see if any of the new columns have missing values. This will tell me if the data that is missing includes any of the incident dates for the mortality dataset. 

In [ ]:
df_with_visits[["NonRecreationVisitors", "RecreationVisitors"]].isna().sum()


There's quite a bit of missing data, so I need to figure out why.

In [ ]:
missing_visits = df_with_visits[
    df_with_visits["NonRecreationVisitors"].isna() |
    df_with_visits["RecreationVisitors"].isna()
]
missing_visits.head()


In [ ]:
missing_visits["parkCode"].nunique()

In [ ]:
missing_visits["parkCode"].value_counts()


In [ ]:
missing_visits["Park Name"].value_counts()


In [ ]:
missing_codes = missing_visits["parkCode"].unique()
missing_codes


In [ ]:
codes_in_visits = visits_df["UnitCode"].unique()
codes_with_data = [code for code in missing_codes if code in codes_in_visits]

codes_with_data


In [ ]:
mismatch_counts

I'm going to see if I made any errors in my manual crosswalk that could be causing this issue with the park codes. I'm going to create a dataframe with that .csv and see if any of the problematic codes are there.

In [ ]:
crosswalk = pd.read_csv("parkname_manual_crosswalk.csv")
crosswalk.head()


In [ ]:
crosswalk["parkCode_manual"] = crosswalk["parkCode_manual"].str.upper()
crosswalk.head()

In [ ]:
matches = crosswalk[crosswalk["parkCode_manual"].isin(mismatch_counts.index)]
matches

In [ ]:
morematches = crosswalk[crosswalk["parkCode_manual"].isin(missing_codes)]
morematches

I found a table that has all the historic codes for sites that may help me find the missing data. I'm going to try to directly download that table of information into Python.

In [ ]:
url = "https://www.nps.gov/articles/000/historic-listing-of-nps-park-codes.htm"
tables = pd.read_html(url)
print(len(tables))          # how many tables pandas found
df_codes = tables[0]        # pick the first one
df_codes.head()

It looks like I got the correct data, but the column names should actually be the data in row 0. I'm going to drop that row and reset the index.

In [ ]:
df_codes.columns = df_codes.iloc[0]
df_codes = df_codes.drop(df_codes.index[0])
df_codes = df_codes.reset_index(drop=True)
df_codes.columns = df_codes.columns.str.strip()
df_codes.head()

In [ ]:
df_codes.shape

I'm going to combine my missing_coes and mismatch_counts but one is an array so I'll turn them both into lists first.

In [ ]:
missing_codes_list = list(missing_codes)
mismatch_counts_list = list(mismatch_counts.index)

And then combine them

In [ ]:
combined_missing = missing_codes_list + mismatch_counts_list
combined_missing


There are duplicates so I'm going to delete them.

In [ ]:
combined_missing = list(set(combined_missing))
combined_missing


I'm still not really sure why these codes are missing data. I'm going to do an API call for one of them to see if there is ANY visitation data available.

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"
params = {
    "unitCodes": "SACR",
    "startYear": 2010,
    "endYear": 2025,
    "format": "json",
}
r = requests.get(BASE, params=params)
print(r.status_code)


In [ ]:
print(r.text[:500])

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"
params = {
    "unitCodes": "SACR",
    "startYear": 2010,
    "endYear": 2025,
    "format": "json",
}
r = requests.get(BASE, params=params)
data = r.json()

# find earliest record
earliest = min([f"{d['Year']}-{d['Month']:02}" for d in data])
earliest


I've been looking at the data on the NPS website and it seems like the codes used in the visitation stats don't match all the official codes. For example, Kings Canyon National Park shows a code of KICA, which isn't anywhere in my historical table or the official park codes table. To make matches, I'm going to download all the names and codes from the park so that I can find more matches for the parks units.

On the NPS searchable page, I can find a drop down menu with all of the parks and park codes as they are listed in the visitation stats. I cannot find a complete listing and the park code is required to call the API so I can't get it from there. Using the inspect tool, I've found the code for the dropdown list that populates with all the park names and codes. It is pretty lengthy, so I'm going to copy it from the inpsect panel and save it in a new .html file called visit_codes.html

I'm going to read that .html file in my notbook, and look the length and the first 500 characters to make sure it's opening correctly.

In [ ]:
with open("visit_codes.html", "r", encoding="utf-8") as f:
    html = f.read()

print(len(html))
print(html[:500])


Now I'm going to use BeautifulSoup to find all the items in the boundlist that I copied. This should be all my parks. I'm going to look at the length to see if it's working correctly. Each item that has a class="x-boundlist-item" should be an individual park, so I'm going to divide those into a list and see how many there are. I'm expcting just a little over 400, because that's how many national park units I know exist.

In [ ]:
soup = BeautifulSoup(html, "html.parser")

items = soup.find_all("li", class_="x-boundlist-item")
print(len(items))
print(items[0:5])

Now that it's divided into each list item instead of a giant string of code, I'm going to put it into rows.

In [ ]:
#create blank place to hold the rows when they're created
rows = []

#create for loop that will go through all of the boundlist items 
for li in items:
    #get just the text that would display withough the rest of the code
    text = li.get_text(strip=True)   

    #I'm going to use a regular expression to seperate the part in parentheses from the rest
    m = re.match(r"^(.*)\(([^)]+)\)$", text)
    if m:
        name = m.group(1).strip()
        code = m.group(2).strip()
        rows.append({"UnitName": name, "UnitCode": code})
    else:
        # in case some weird row doesn't match that pattern
        rows.append({"UnitName": text, "UnitCode": None})


In [ ]:
len(df_visit_codes)

In [ ]:
df_visit_codes = pd.DataFrame(rows)

df_visit_codes.head()



In [ ]:
df_visit_codes.tail(

)

Since my loop indicated that rows missing park codes or in a different format should record visitsParkCode as "None", I'm going to count how many rows have None in that column.

In [ ]:
(df_visit_codes["UnitCode"] == "None").sum()


Now that I have this complete listing for the visitor statistics API, I'm going to save it to a new .csv

In [ ]:
df_visit_codes.to_csv("../data/visit_codes.csv", index=False)


I'll use the API to call for all the parks available in the list and obtain all available visitor data, which I will then map to my dataset.

In [ ]:
BASE = "https://irmaservices.nps.gov/v3/rest/stats/visitation"

all_codes = (
    df_visit_codes["UnitCode"]
    .dropna()
    .astype(str)
    .str.upper()
    .unique()
    .tolist()
)

def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

CODES_PER_CALL = 50

all_results = []

with requests.Session() as s:
    for chunk in chunk_list(all_codes, CODES_PER_CALL):
        params = {
            "unitCodes": ",".join(chunk),
            "startMonth": 1,
            "startYear": 2007,
            "endMonth": 6,
            "endYear": 2024,
            "format": "json",
        }
        r = s.get(BASE, params=params, headers={"Accept": "application/json"}, timeout=30)
        r.raise_for_status()
        data = r.json()
        all_results.extend(data)

all_visits_df = pd.DataFrame(all_results)


In [ ]:
all_visits_df.head()

In [ ]:
all_visits_df.shape

In [ ]:
visits_df.head()

In [ ]:
visits_df.shape

The columns are the same. If my new data includes everything in my old data, I should have the same number of duplicate rows as I do rows in visits_df. 

In [ ]:
# merge on all columns to find exact duplicates between dataframes
shared = all_visits_df.merge(visits_df, how="inner")

len(shared)


Since that was what expected, I am going to overwrite the NPS_visitation_data.csv to hold all the data.

In [ ]:
all_visits_df.to_csv("../data/NPS_visitation_data.csv", index=False)


I'm going to add my new visitation unit codes to my mortality dataset so that I can put all the new data in the correct places.

To efficiently determine the best matches for each park, I utilized ChatGPT. I uploaded df_with_visits.csv, as well as visit_codes.csv, and provide my list of parks without matches. I prompted ChatGPT to use information about NPS park structures, historical locations and unit codes, and geographic locations to determin the best match for incidents that didn't have data. That file is uploaded in the data folder as corrected_park_codes.csv. I'm going to use this code to help match my new visitation data.

In [ ]:
df_corrected_park_codes = pd.read_csv("../Data/corrected_park_codes.csv")

df_corrected_park_codes.head()


In [ ]:
df_corrected_park_codes.shape

I'm going to change some of the column names that ChatGPT generated to make it easier to utilize this list.

In [ ]:
df_corrected_park_codes = df_corrected_park_codes.rename(
    columns={"Queried parkCode": "parkCode",
             "Matched visitsParkName": "UnitName",
             "Matched visitsParkCode": "UnitCode"})

df_corrected_park_codes.head()

In [ ]:
df_with_visits.columns

In [ ]:
df_corrected_park_codes.columns

In [ ]:
df_with_visits["UnitCode"].isnull().sum()

Now I'm going to merge this info to df_with_visits df.

In [ ]:
# 0) make sure there are no sneaky spaces in the corrected df
#df_corrected_park_codes.columns = df_corrected_park_codes.columns.str.strip()

# 1) now do the merge (this should work now)
df_with_visits = df_with_visits.merge(
    df_corrected_park_codes[["Park Name", "UnitCode", "UnitName"]],
    how="left",
    left_on="fullName_official",
    right_on="Park Name",
    suffixes=("", "_corr")   # new cols from corrected df will be UnitCode_corr, UnitName_corr
)

# 2) fill ONLY where original was NaN, using the corrected ones
df_with_visits["UnitCode"] = df_with_visits["UnitCode"].fillna(df_with_visits["UnitCode_corr"])
df_with_visits["UnitName"] = df_with_visits["UnitName"].fillna(df_with_visits["UnitName_corr"])

# 3) drop helper columns
df_with_visits = df_with_visits.drop(columns=["UnitCode_corr", "UnitName_corr", "Park Name_corr"])

df_with_visits.head()


Now I'll rerun my count to confirm codes were added.

In [ ]:
df_with_visits["UnitCode"].isnull().sum()

The number decreased so some were added, but I'm still missing rows. I want to look at the unique values and see which parks still aren't complete.

In [ ]:
df_with_visits[df_with_visits["UnitCode"].isnull()]["parkCode"].unique()


Now I'm going to check if any of those parkCodes have any rows with a UnitCode, which would indicate that some data was downloade in the prior API call, but not all.

In [ ]:
#Get the parkCodes where UnitCode is null
missing_codes = df_with_visits.loc[df_with_visits["UnitCode"].isnull(), "parkCode"].unique()

#Filter rows where parkCode is one of those, and UnitCode is NOT null
partial_matches = df_with_visits.loc[
    df_with_visits["parkCode"].isin(missing_codes) & df_with_visits["UnitCode"].notnull(),
    ["parkCode", "UnitCode"]
]

#View the unique parkCodes that have both missing and present UnitCodes
partial_matches["parkCode"].unique()


I still have some without matches. I again utilized ChatGPT to make matches in a new file, gpt_code_matches.csv

In [ ]:
df_more_corrected_park_codes = pd.read_csv("../Data/gpt_code_matches.csv")

df_more_corrected_park_codes.head()


In [ ]:
df_more_corrected_park_codes = df_more_corrected_park_codes.rename(
    columns={"Queried parkCode": "parkCode",
             "Matched visitsParkName": "UnitName",
             "Matched visitsParkCode": "UnitCode"})

df_more_corrected_park_codes.head()

In [ ]:
# merge in extra matches from gpt_code_matches, using parkCode as the key
df_with_visits = df_with_visits.merge(
    df_more_corrected_park_codes[["parkCode", "UnitCode", "UnitName"]],
    how="left",
    on="parkCode",
    suffixes=("", "_gpt")
)

# fill only missing values
df_with_visits["UnitCode"] = df_with_visits["UnitCode"].fillna(df_with_visits["UnitCode_gpt"])
df_with_visits["UnitName"] = df_with_visits["UnitName"].fillna(df_with_visits["UnitName_gpt"])

# drop helper cols
df_with_visits = df_with_visits.drop(columns=["UnitCode_gpt", "UnitName_gpt"])


In [1750]:
df_with_visits.head()

,date,parkName,causeGroup,sex,activity,ageRange,ageMin,ageMax,ageMid,parkCode,officialName,intent,cause,Month,Year,NonRecreationVisitors,RecreationVisitors,UnitCode,UnitName
0,2007-01-01,Glen Canyon National Recreation Area,Undetermined,Male,Not Reported,65+,65.0,NaN,NaN,GLCA,Glen Canyon National Recreation Area,undetermined,undetermined,1,2007,899.0,29707.0,GLCA,Glen Canyon NRA
1,2007-01-22,Golden Gate National Recreation Area,Drowning,Male,Vessel Related,Unknown,NaN,NaN,NaN,GOGA,Golden Gate National Recreation Area,unintentional,drowning,1,2007,0.0,992940.0,GOGA,Golden Gate NRA
2,2007-01-22,Golden Gate National Recreation Area,Undetermined,Male,Vessel Related,Unknown,NaN,NaN,NaN,GOGA,Golden Gate National Recreation Area,undetermined,undetermined,1,2007,0.0,992940.0,GOGA,Golden Gate NRA
3,2007-01-29,Natchez Trace Parkway,Motor Vehicle Crash,Female,Driving,15-24,15.0,24.0,19.5,NATR,Natchez Trace Parkway,unintentional,motor vehicle crash,1,2007,577583.0,486268.0,NATR,Natchez Trace PKWY
4,2007-01-29,Natchez Trace Parkway,Motor Vehicle Crash,Female,Driving,45-54,45.0,54.0,49.5,NATR,Natchez Trace Parkway,unintentional,motor vehicle crash,1,2007,577583.0,486268.0,NATR,Natchez Trace PKWY


In [1752]:
df_with_visits["UnitCode"].isnull().sum()

np.int64(11)

In [1748]:
df_with_visits[df_with_visits["UnitCode"].isnull()]["parkCode"].unique()


array(['APPA', 'AMME'], dtype=object)

This is what I'm expecting to see. NPS data is not available for either the Appalachian Trail or the American Memorial Park in Saipan.

Now I'm going to do some housecleaning on my mortality dataset so that I can appropriately merge the visitation data. I'll start by looking at the header to make sure I'm working in the correct place with the correct fields.

In [ ]:
df_with_visits.head()

First I'm going to clean up the column names to be clear, concise, and consistent. I'll start by looking at the list of column names.

In [ ]:
df_with_visits.columns

In [ ]:
df_with_visits = df_with_visits.rename(
    columns={"Incident Date": "date",
             "Park Name": "parkName",
             "Cause of Death Group \n(Used in the NPS Mortality Dashboard) ":"causeGroup", 
             "Sex": "sex",
             "Activity": "activity",
             "age_range_clean": "ageRange", 
             "age_min": "ageMin", 
             "age_max": "ageMax",
             "age_mid": "ageMid",
             "fullName_official": "officialName",
              "Intent_norm": "intent", 
              "Cause_norm": "cause"}
)

df_with_visits.head()


Now we'll merge the visitation data to the fields that now have parkCodes.

In [ ]:
all_visits_df.columns

In [ ]:
# merge in extra matches from gpt_code_matches, using parkCode as the key
df_with_visits = df_with_visits.merge(
    all_visits_df[["UnitCode", "Month", "Year", "NonRecreationVisitors", "RecreationVisitors"]],
    how="left",
    on=["UnitCode", "Month", "Year"],
    suffixes=("", "_visit")
)

# fill only where you were missing values
df_with_visits["NonRecreationVisitors"] = df_with_visits["NonRecreationVisitors"].fillna(df_with_visits["NonRecreationVisitors_visit"])
df_with_visits["RecreationVisitors"] = df_with_visits["RecreationVisitors"].fillna(df_with_visits["RecreationVisitors_visit"])

# drop helper cols
df_with_visits = df_with_visits.drop(columns=["NonRecreationVisitors_visit", "RecreationVisitors_visit"])

df_with_visits.head()


In [ ]:
df_with_visits.columns

In [ ]:
df_with_visits.isna().sum()

In [1768]:
df_with_visits.loc[df_with_visits["RecreationVisitors"].isna(), "parkCode"].unique()

array(['BAWA', 'APPA', 'SACR', 'PAGR', 'FRST', 'AMME'], dtype=object)

We should only be missing all of the data for the parks we previously identified, Appalachian Trail and American Memorial Park. 

In [1770]:
df_with_visits.loc[
    df_with_visits["RecreationVisitors"].isna() & df_with_visits["UnitCode"].isna(),
    ["officialName", "parkCode", "UnitCode", "RecreationVisitors"]
]


,officialName,parkCode,UnitCode,RecreationVisitors
563,Appalachian National Scenic Trail,APPA,NaN,NaN
1470,Appalachian National Scenic Trail,APPA,NaN,NaN
2619,Appalachian National Scenic Trail,APPA,NaN,NaN
2704,Appalachian National Scenic Trail,APPA,NaN,NaN
2763,Appalachian National Scenic Trail,APPA,NaN,NaN
2825,Appalachian National Scenic Trail,APPA,NaN,NaN
3154,Appalachian National Scenic Trail,APPA,NaN,NaN
3222,Appalachian National Scenic Trail,APPA,NaN,NaN
4214,American Memorial Park,AMME,NaN,NaN
4421,Appalachian National Scenic Trail,APPA,NaN,NaN


In [1769]:
df_with_visits.loc[df_with_visits["parkCode"] == "BAWA", ["UnitCode", "RecreationVisitors"]]


,UnitCode,RecreationVisitors
138,NACA,NaN
142,NACA,NaN
181,NACA,NaN
186,NACA,NaN
296,NACA,NaN
...,...,...
4530,NACA,NaN
4551,NACA,NaN
4565,NACA,NaN
4599,NACA,NaN


I did research on NPS websites to determine if there may be any reasons for missing data for the remaining parks, BAWA, SACR, PAGR, FRST, that have some but not all of the data. My findings are:

BAWA is a parkway and has gone many retoolings in how visitation data is collected. Sometimes the data is even collected by the FHWA in conjuction with NPS. I wan't able to find any alternative sources for the missing data. We will have to deal with these on a case by case basis to determine how lack of data may affect normalization.

SACR - This is an island site that is run in conjunction with Canada. It is a part of the Acadia region, and the data may be stored under that park (ACAD). I'll see what I have from ACAD in my full visitation data file.

PAGR was established in 2011, so there should not be data before then, and the earlier years may be incomplete/inaccurate. I will double check the earliest missing dates to confirm this is the only problem.

FRST - Park began as National Monument on 25Mar2013 and became National Historic Park in 2014. Data should begin in 2013/2014. Will confirm.



In [1771]:
all_visits_df.head()

,Month,NonRecreationVisitors,RecreationVisitors,UnitCode,UnitName,Year
0,1,0,4960,ABLI,Abraham Lincoln Birthplace NHP,2007
1,2,0,5877,ABLI,Abraham Lincoln Birthplace NHP,2007
2,3,0,9868,ABLI,Abraham Lincoln Birthplace NHP,2007
3,4,0,17900,ABLI,Abraham Lincoln Birthplace NHP,2007
4,5,0,21277,ABLI,Abraham Lincoln Birthplace NHP,2007


In [ ]:
all_visits_df.head()

In [1776]:
missing_rows = ["NACA", "SACR", "PAGR", "FRST"]

# filter just those
subset = all_visits_df[all_visits_df["UnitCode"].isin(missing_rows)].copy()

subset["Date"] = pd.to_datetime(
    subset["Year"].astype(str) + "-" + subset["Month"].astype(str) + "-01"
)

# 2) get earliest and latest row per park
first_rows = subset.loc[
    subset.groupby("UnitCode")["Date"].idxmin(),
    ["UnitCode", "Year", "Month", "Date", "RecreationVisitors"]
].rename(columns={
    "Year": "EarliestYear",
    "Month": "EarliestMonth",
    "Date": "EarliestDate",
    "RecreationVisitors": "EarliestVisitors"
})

last_rows = subset.loc[
    subset.groupby("UnitCode")["Date"].idxmax(),
    ["UnitCode", "Year", "Month", "Date", "RecreationVisitors"]
].rename(columns={
    "Year": "LatestYear",
    "Month": "LatestMonth",
    "Date": "LatestDate",
    "RecreationVisitors": "LatestVisitors"
})

first_last = first_rows.merge(last_rows, on="UnitCode")
print("=== First and last per park ===")
print(first_last)


gap_records = []

for code, group in subset.groupby("UnitCode"):
    group = group.sort_values("Date")
    # full monthly range from first to last
    full_range = pd.date_range(group["Date"].min(), group["Date"].max(), freq="MS")
    # which dates are actually present
    present_dates = set(group["Date"])
    # find missing ones
    missing = [d for d in full_range if d not in present_dates]
    if missing:
        gap_records.append({
            "UnitCode": code,
            "MissingDates": missing
        })

gaps_df = pd.DataFrame(gap_records)
gaps_df.head()

=== First and last per park ===
  UnitCode  EarliestYear  EarliestMonth EarliestDate  EarliestVisitors  \
0     FRST          2023              1   2023-01-01                 0   
1     PAGR          2016              1   2016-01-01               248   
2     SACR          2013              1   2013-01-01                 0   

   LatestYear  LatestMonth LatestDate  LatestVisitors  
0        2024            6 2024-06-01           28962  
1        2024            6 2024-06-01           31823  
2        2024            6 2024-06-01            1854  


""


In [1773]:
all_visits_df.loc[all_visits_df["UnitCode"] == "NACA"].head()


,Month,NonRecreationVisitors,RecreationVisitors,UnitCode,UnitName,Year


In [1774]:
df_with_visits.loc[df_with_visits["parkCode"] == "BAWA"].head()


,date,parkName,causeGroup,sex,activity,ageRange,ageMin,ageMax,ageMid,parkCode,officialName,intent,cause,Month,Year,NonRecreationVisitors,RecreationVisitors,UnitCode,UnitName
138,2007-12-08,Baltimore-Washington Parkway,Motor Vehicle Crash,Not Reported,Driving,Unknown,NaN,NaN,NaN,BAWA,Baltimore-Washington Parkway,unintentional,motor vehicle crash,12,2007,NaN,NaN,NACA,National Capital Parks Combined
142,2007-12-29,Baltimore-Washington Parkway,Motor Vehicle Crash,Male,Not Reported,35-44,35.0,44.0,39.5,BAWA,Baltimore-Washington Parkway,unintentional,motor vehicle crash,12,2007,NaN,NaN,NACA,National Capital Parks Combined
181,2008-05-23,Baltimore-Washington Parkway,Motor Vehicle Crash,Male,Driving,Unknown,NaN,NaN,NaN,BAWA,Baltimore-Washington Parkway,unintentional,motor vehicle crash,5,2008,NaN,NaN,NACA,National Capital Parks Combined
186,2008-05-31,Baltimore-Washington Parkway,Motor Vehicle Crash,Female,Driving,25-34,25.0,34.0,29.5,BAWA,Baltimore-Washington Parkway,unintentional,motor vehicle crash,5,2008,NaN,NaN,NACA,National Capital Parks Combined
296,2009-02-10,Baltimore-Washington Parkway,Motor Vehicle Crash,Male,Walking,35-44,35.0,44.0,39.5,BAWA,Baltimore-Washington Parkway,unintentional,motor vehicle crash,2,2009,NaN,NaN,NACA,National Capital Parks Combined


In [1777]:
bawa_rows = df_with_visits[df_with_visits["parkCode"] == "BAWA"]

bawa_rows[["Month", "Year", "RecreationVisitors"]].sort_values(["Year", "Month"])


,Month,Year,RecreationVisitors
138,12,2007,NaN
142,12,2007,NaN
181,5,2008,NaN
186,5,2008,NaN
296,2,2009,NaN
...,...,...,...
4530,9,2023,NaN
4551,10,2023,NaN
4565,10,2023,NaN
4599,2,2024,NaN


In [1780]:
print(f"Total BAWA rows: {total_bawa}")
print(f"Rows with RecreationVisitors data: {count_with_data}")
print(f"Rows missing RecreationVisitors data: {total_bawa - count_with_data}")


Total BAWA rows: 83
Rows with RecreationVisitors data: 0
Rows missing RecreationVisitors data: 83


In [1786]:
# Filter for UnitCode ACAD
acad_rows = all_visits_df[all_visits_df["UnitCode"].astype(str).str.upper().eq("ACAD")]

# Count rows that have RecreationVisitors data (non-null)
yesacad = acad_rows["RecreationVisitors"].notna().sum()

# Count rows that are missing RecreationVisitors data (null)
noacad = acad_rows["RecreationVisitors"].isna().sum()

# Total rows for ACAD
total = len(acad_rows)

print(f"Total acad rows: {total}")
print(f"Rows with RecreationVisitors data: {yesacad}")
print(f"Rows missing RecreationVisitors data: {noacad}")



Total acad rows: 210
Rows with RecreationVisitors data: 210
Rows missing RecreationVisitors data: 0


In [1787]:
# Filter for ACAD and SACR rows
acad_sacr = all_visits_df[all_visits_df["UnitCode"].isin(["ACAD", "SACR"])]

# Find which RecreationVisitors values appear in both parks
shared_values = (
    acad_sacr.groupby(["RecreationVisitors"])["UnitCode"]
    .nunique()
    .reset_index(name="unit_count")
    .query("unit_count > 1")
)

# Show which values those are
if shared_values.empty:
    print("No RecreationVisitors values are shared between ACAD and SACR.")
else:
    print("RecreationVisitors values shared between ACAD and SACR:")
    display(shared_values)


No RecreationVisitors values are shared between ACAD and SACR.


In [1784]:
# Filter for ACAD and SACR rows
acad_sacr = all_visits_df[all_visits_df["UnitCode"].isin(["ACAD", "SACR"])]

# Group by UnitCode and RecreationVisitors to find duplicates
duplicates = (
    acad_sacr.groupby(["UnitCode", "RecreationVisitors"])
    .size()
    .reset_index(name="count")
    .query("count > 1")
)

# Display results
if duplicates.empty:
    print("No duplicate RecreationVisitors values found for ACAD or SACR.")
else:
    print("Duplicate RecreationVisitors values found:")
    display(duplicates)


Duplicate RecreationVisitors values found:


,UnitCode,RecreationVisitors,count
210,SACR,0,65


In [1785]:
all_visits_df.loc[
    all_visits_df["UnitCode"].eq("SACR") & all_visits_df["RecreationVisitors"].eq(0),
    ["Year", "Month", "NonRecreationVisitors"]
].sort_values(["Year", "Month"])


,Year,Month,NonRecreationVisitors
63112,2013,1,0
63113,2013,2,0
63114,2013,3,0
63115,2013,4,0
63122,2013,11,0
...,...,...,...
63234,2023,3,0
63243,2023,12,0
63244,2024,1,0
63245,2024,2,0


In [1792]:
all_visits_df.loc[
    all_visits_df["UnitCode"].eq("SACR"),
    ["Year", "Month", "RecreationVisitors"]
].sort_values(["Year", "Month"])


,Year,Month,RecreationVisitors
63112,2013,1,0
63113,2013,2,0
63114,2013,3,0
63115,2013,4,0
63116,2013,5,1177
...,...,...,...
63245,2024,2,0
63246,2024,3,0
63247,2024,4,682
63248,2024,5,1174


In [1791]:
all_visits_df.loc[
    all_visits_df["UnitCode"].eq("ACAD"),
    ["Year", "Month", "RecreationVisitors"]
].sort_values(["Year", "Month"])


,Year,Month,RecreationVisitors
210,2007,1,12415
211,2007,2,11068
212,2007,3,19220
213,2007,4,61775
214,2007,5,132152
...,...,...,...
415,2024,2,14071
416,2024,3,25696
417,2024,4,95165
418,2024,5,320885


I know that SACR is part of ACAD, but I also know that SACR is closed in winter months. When I look at overlapping results between the two, it doesn't seem like ACAD would be a fair or accurate representation of the visitors at the SACR site because there are many winter visitors in ACAD while SACR is closed. I will not fill any rows with data from ACAD, despita the organizational affiliation, because it would be more damaging to my analysis to have inaccurate data than incomplete data.

In [1810]:
# Filter for SACR rows
sacr_rows = df_with_visits[df_with_visits["UnitCode"].astype(str).str.upper().eq("SACR")]

# Count missing values in RecreationVisitors and NonRecreationVisitors
missing_recreation = sacr_rows["RecreationVisitors"].isna().sum()
missing_nonrecreation = sacr_rows["NonRecreationVisitors"].isna().sum()

# Total number of SACR rows for context
total_sacr = len(sacr_rows)

print(f"{total_sacr}")
print(f"{missing_recreation}")

5
0


In [1794]:
# Show rows missing RecreationVisitors
missing_recreation = sacr_rows[sacr_rows["RecreationVisitors"].isna()][["Year", "Month", "RecreationVisitors", "NonRecreationVisitors"]]

# Show rows missing NonRecreationVisitors
missing_nonrecreation = sacr_rows[sacr_rows["NonRecreationVisitors"].isna()][["Year", "Month", "RecreationVisitors", "NonRecreationVisitors"]]

print("Rows missing RecreationVisitors:")
display(missing_recreation.sort_values(["Year", "Month"]))

print("\nRows missing NonRecreationVisitors:")
display(missing_nonrecreation.sort_values(["Year", "Month"]))

Rows missing RecreationVisitors:


,Year,Month,RecreationVisitors,NonRecreationVisitors
879,2012,8,NaN,NaN
910,2012,10,NaN,NaN



Rows missing NonRecreationVisitors:


,Year,Month,RecreationVisitors,NonRecreationVisitors
879,2012,8,NaN,NaN
910,2012,10,NaN,NaN


I have several years of complete data for 2013 on, so I feel comfortable trying to use those years of data to approximate the missing information for those two rows. I believe I will likely be able to get a close approximation and the effect of using a number that may be slightly inaccurate will minimal on the overall analysis.

In [1796]:

#Calculate the yearly averages
yearly_avg = (
    sacr_train.groupby("Year")[["RecreationVisitors", "NonRecreationVisitors"]]
    .mean()
    .sort_index()
)

yearly_avg.head()


,RecreationVisitors,NonRecreationVisitors
Year,,
2013,892.416667,0.0
2014,965.666667,0.0
2015,1046.416667,0.0
2016,1154.750000,0.0
2017,989.333333,0.0


In [ ]:
yoy = yearly_avg.pct_change().add(1)

yoy.head(10)

,RecreationVisitors,NonRecreationVisitors
Year,,
2013,NaN,NaN
2014,1.082080,NaN
2015,1.083621,NaN
2016,1.103528,NaN
2017,0.856751,NaN
2018,1.010866,NaN
2019,0.967669,NaN
2020,0.595367,NaN
2021,1.489586,NaN


I believe that the earlier year in this series of data is more fairly representative of the change that likely occured in my missing years because those value points are reasonably consistent in representing 8-10% YOY growth. I will use only those 3 data points to estimate my missing values to avoid outliers toward the end of the dataset (such as 2020/pandemic year) skewing what appears to have been a stable period of growth. I also can see that NonRecreationVisitors is 0 in all avaialable fields, so I'll ultimately fill 0 for the missing fields, as well.

In [1801]:
#Filter for the first 3 non-null points (2014–2016)
stable_years = yoy.loc[yoy.index.isin([2014, 2015, 2016])]

#Calculate average growth for those years only
avg_growth = stable_years["RecreationVisitors"].mean(skipna=True)

avg_growth

np.float64(1.0897431309964691)

Now I'll calculate the monthly averages from the stable rows only.

In [1805]:
# Filter SACR rows from all_visits_df
sacr_visits = all_visits_df[all_visits_df["UnitCode"] == "SACR"].copy()

# Combine year and month fields to make numeric value that can be compared in series order
sacr_visits["Year"] = sacr_visits["Year"].astype(int)
sacr_visits["Month"] = sacr_visits["Month"].astype(int)
sacr_visits["year_month"] = sacr_visits["Year"] * 100 + sacr_visits["Month"]

# Use only stable early years (2013–2016) for training
early = sacr_visits[sacr_visits["Year"].between(2013, 2016)].copy()

# Add column with calculated monthly averages
monthly_avg = (
    early.groupby("Month")[["RecreationVisitors", "NonRecreationVisitors"]]
    .mean()
    .rename(columns={
        "RecreationVisitors": "RecreationVisitors_month_avg",
        "NonRecreationVisitors": "NonRecreationVisitors_month_avg"
    })
)
monthly_avg.head(12)


,RecreationVisitors_month_avg,NonRecreationVisitors_month_avg
Month,,
1,0.00,0.0
2,0.00,0.0
3,0.00,0.0
4,0.00,0.0
5,1086.00,0.0
6,1704.25,0.0
7,2887.75,0.0
8,2974.50,0.0
9,2272.75,0.0


In [1807]:
# Estimate 2012 August & October values using monthly averages and early-period growth
target_months = [8, 10]
estimates_2012 = {}
for m in target_months:
    rec_val = monthly_avg.loc[m, "RecreationVisitors_month_avg"] / avg_growth_rec
    # Round to whole people
    rec_val = int(round(rec_val))

    estimates_2012[m] = {
        "RecreationVisitors": rec_val,
        "NonRecreationVisitors": 0
    }

# Fill nulls in df_with_visits
for m in target_months:
    mask = (
        (df_with_visits["UnitCode"] == "SACR") &
        (df_with_visits["Year"] == 2012) &
        (df_with_visits["Month"] == m)
    )

    if m in estimates_2012:
        df_with_visits.loc[
            mask & df_with_visits["RecreationVisitors"].isna(),
            "RecreationVisitors"
        ] = estimates_2012[m]["RecreationVisitors"]

        df_with_visits.loc[
            mask & df_with_visits["NonRecreationVisitors"].isna(),
            "NonRecreationVisitors"
        ] = 0

# Inspect results
df_with_visits.loc[
    (df_with_visits["UnitCode"] == "SACR") &
    (df_with_visits["Year"] == 2012) &
    (df_with_visits["Month"].isin(target_months)),
    ["UnitCode", "Year", "Month", "RecreationVisitors", "NonRecreationVisitors"]
]


,UnitCode,Year,Month,RecreationVisitors,NonRecreationVisitors
879,SACR,2012,8,2977.0,0.0
910,SACR,2012,10,1254.0,0.0


Sanity check: I'll rerun this to see if there are still any missing rows.

In [1811]:
# Filter for SACR rows
sacr_rows = df_with_visits[df_with_visits["UnitCode"].astype(str).str.upper().eq("SACR")]

# Count missing values in RecreationVisitors and NonRecreationVisitors
missing_recreation = sacr_rows["RecreationVisitors"].isna().sum()
missing_nonrecreation = sacr_rows["NonRecreationVisitors"].isna().sum()

# Total number of SACR rows for context
total_sacr = len(sacr_rows)

print(f"{total_sacr}")
print(f"{missing_recreation}")

5
0
